In [32]:
import glob 
import numpy as np
from keras.layers import LSTM,Dense,Embedding,Dropout
from keras.models import Sequential
from keras.utils import np_utils 
from music21 import converter,instrument,note,chord
from pickle import dump   #used to save file 

In [23]:
def get_notes():
    #function created to get notes from the files dataset downloaded
    notes=[]
    for file in glob.glob("Piano-midi.de/train/*.mid"):
        midi=converter.parse(file)
        #Using that stream object we get a list of all the notes and chords in the file. 
        print(file)
        notes_to_pass=None
        parts=instrument.partitionByInstrument(midi)
        if parts:#file has instrument part
            notes_to_pass=parts.parts[0].recurse()
        else:#file has notes in flat format
            notes_to_pass=midi.flat.notes
        for elements in notes_to_pass:
            if isinstance(elements,note.Note):
                notes.append(str(elements.pitch))
            elif isinstance(elements,chord.Chord):
                notes.append('.'.join(str(n) for n in elements.normalOrder))
    # saving the notes we have created 
    with open("notes","wb") as filepath:
        dump(notes,filepath)
    return notes

# what i have done above 

We start by loading each file into a Music21 stream object using the converter.parse(file) function. Using that stream object we get a list of all the notes and chords in the file. We append the pitch of every note object using its string notation since the most significant parts of the note can be recreated using the string notation of the pitch. And we append every chord by encoding the id of every note in the chord together into a single string, with each note being separated by a dot. These encodings allows us to easily decode the output generated by the network into the correct notes and chords.

# next

we have to create input sequences for the network and their respective outputs. The output for each input sequence will be the first note or chord that comes after the sequence of notes in the input sequence in our list of notes.

In [30]:
def create_seq(notes,n_vocab):
    length=50
    pitchname=sorted(set(pitch for pitch in notes))
    #mapping each node to int
    note_to_int=dict((note,number) for number,note in enumerate(pitchname))
    input_seq=[]
    output_seq=[]
    for i in range(0,len(notes)-length,1):
        in_seq=notes[i:i+length]
        out_seq=notes[i+length]
        input_seq.append([note_to_int[char] for char in in_seq])
        output_seq.append(note_to_int[out_seq])
    n_pattern=len(input_seq)
    #input_seq=np.array(input_seq)
    #output_seq=np.array(output_seq)

    #reshaping the input to amke it compatible with lstm
    input_seq=np.reshape(input_seq,(n_pattern,length,1))
    #normalizing
    input_seq=input_seq/float(n_vocab)
    output_seq=np_utils.to_categorical(output_seq)
    return (input_seq,output_seq)

In [26]:
notes=get_notes()

Piano-midi.de/train/islamei.mid
Piano-midi.de/train/ty_mai.mid
Piano-midi.de/train/ty_april.mid
Piano-midi.de/train/chpn-p2.mid
Piano-midi.de/train/grieg_kobold.mid
Piano-midi.de/train/schub_d960_1.mid
Piano-midi.de/train/alb_se4.mid
Piano-midi.de/train/bach_850.mid
Piano-midi.de/train/liz_et1.mid
Piano-midi.de/train/grieg_waechter.mid
Piano-midi.de/train/mendel_op62_3.mid
Piano-midi.de/train/muss_5.mid
Piano-midi.de/train/deb_pass.mid
Piano-midi.de/train/alb_se1.mid
Piano-midi.de/train/bor_ps1.mid
Piano-midi.de/train/waldstein_1.mid
Piano-midi.de/train/alb_esp4.mid
Piano-midi.de/train/chpn_op33_4.mid
Piano-midi.de/train/waldstein_3.mid
Piano-midi.de/train/liz_et4.mid
Piano-midi.de/train/chpn_op33_2.mid
Piano-midi.de/train/muss_6.mid
Piano-midi.de/train/schub_d760_2.mid
Piano-midi.de/train/muss_7.mid
Piano-midi.de/train/burg_gewitter.mid
Piano-midi.de/train/scn15_6.mid
Piano-midi.de/train/gra_esp_2.mid
Piano-midi.de/train/hay_40_1.mid
Piano-midi.de/train/alb_esp1.mid
Piano-midi.de/trai

In [31]:
n_vocab=len(set(notes))
input_seq,output_seq=create_seq(notes,n_vocab)

In [37]:
#cretaing deep learning model 
model=Sequential()
model.add(LSTM(256,input_shape=(input_seq.shape[1],input_seq.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation="softmax"))
model.compile(loss="categorical_crossentropy",optimizer="rmsprop")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 50, 256)           264192    
_________________________________________________________________
dropout_10 (Dropout)         (None, 50, 256)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 50, 512)           1574912   
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 512)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
__________

In [38]:
from keras.callbacks import ModelCheckpoint

In [40]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,save_best_only=True,mode='min')    
callbacks_list = [checkpoint]
model.fit(input_seq,output_seq, epochs=200, batch_size=64, callbacks=callbacks_list)

Epoch 1/200
148311/148311 [==============================] - 3695s 25ms/step - loss: 4.7483
Epoch 2/200
148311/148311 [==============================] - 3512s 24ms/step - loss: 4.6375
Epoch 3/200
148311/148311 [==============================] - 3696s 25ms/step - loss: 4.6381
Epoch 4/200
 16192/148311 [==>...........................] - ETA: 1:00:19 - loss: 4.6065

KeyboardInterrupt: 